In [22]:
import pandas as pd
import locale
from holidays import country_holidays
import matplotlib.pyplot as plt
import seaborn as sns
import requests
#from tmdbv3api import TMDb, Movie
import os
from dotenv import load_dotenv
from tqdm.auto import tqdm
load_dotenv()

True

In [23]:
# Stile dei grafici
sns.set(style="whitegrid")

# Mappa dei giorni in italiano (corretta e sicura)
giorni_settimana = {
    "Monday": "lunedì",
    "Tuesday": "martedì",
    "Wednesday": "mercoledì",
    "Thursday": "giovedì",
    "Friday": "venerdì",
    "Saturday": "sabato",
    "Sunday": "domenica"
}

# Percorso assoluto basato sulla posizione del file corrente
# Definisci la variabile 'path' con il percorso corretto del file
path = os.path.join('input', 'dati_cinema_originali.csv')
file_path = path

In [24]:
# Caricamento con encoding
df = pd.read_csv(path, encoding="latin1", sep=";", dayfirst=True, on_bad_lines="skip")
print(df.head())

# Conversione della data
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y', errors='coerce')

# Combina data + ora reale
df['datetime'] = pd.to_datetime(df['date'].dt.date.astype(str) + ' ' + df['time'], errors='coerce')

# Ordina per data
df = df.sort_values("datetime").reset_index(drop=True)


# Estrazione e conversione
df["giorno_settimana_en"] = df["datetime"].dt.day_name()
df["giorno_settimana"] = df["giorno_settimana_en"].map(giorni_settimana)


# Pulizia colonna intermedia
df.drop(columns=["giorno_settimana_en"], inplace=True)


# Mese, anno, ora
df["mese"] = df["datetime"].dt.month
df["anno"] = df["datetime"].dt.year
df["ora"] = df["datetime"].dt.hour


# Fascia oraria
def get_fascia_oraria(ora):
    if ora == 10:
        return "mattina"
    elif 15 <= ora <= 17:
        return "pomeriggio"
    elif 20 <= ora <= 22:
        return "sera"
    else:
        return "altro"  # in caso ci fossero altri orari imprevisti

df["fascia_oraria"] = df["ora"].apply(get_fascia_oraria)


# Funzione per classificare le stagioni
def get_stagione(mese):
    if mese in [12, 1, 2]:
        return "inverno"
    elif mese in [3, 4, 5]:
        return "primavera"
    elif mese in [6, 7, 8]:
        return "estate"
    else:
        return "autunno"

df["stagione"] = df["mese"].apply(get_stagione)

# Weekend: True se sabato o domenica
df["weekend"] = df["giorno_settimana"].isin(["venerdì", "sabato", "domenica"])

# Anni unici presenti nel dataset
anni_presenti = df["anno"].unique()

# Festività italiane ufficiali per quegli anni
festivita_italiane = country_holidays("IT", years=anni_presenti)

# Colonna con nome della festività (NaN se non è una festività)
df["festività"] = df["datetime"].dt.date.map(festivita_italiane)
# Sostituisce i valori NaN con 'Nessuna'
df["festività"] = df["festività"].fillna("Nessuna")


df['total'] = df[['full_price', 'reduced', 'free']].sum(axis=1)
df['full_price'] = df['full_price'].fillna(0)
df['reduced'] = df['reduced'].fillna(0)
df['free'] = df['free'].fillna(0)

df.sample(5)

         date   time            title  full_price  reduced  free  total
0  14/09/2019  20:30      Il Re Leone        83.0    101.0   0.0    184
1  15/09/2019  17:00      Il Re Leone        53.0     67.0   1.0    121
2  15/09/2019  20:30      Il Re Leone        35.0     22.0   0.0     57
3  22/09/2019  17:00      Il Re Leone       114.0    127.0  13.0    254
4  28/09/2019  20:30  IT - Capitolo 2        75.0    123.0   0.0    198


,date,time,title,full_price,reduced,free,total,datetime,giorno_settimana,mese,anno,ora,fascia_oraria,stagione,weekend,festività
545,2025-01-05,20:30,UNA NOTTE A NEW YORK,11.0,17.0,1.0,29.0,2025-01-05 20:30:00,domenica,1,2025,20,sera,inverno,True,Nessuna
381,2024-01-06,17:00,Wish,53.0,85.0,12.0,150.0,2024-01-06 17:00:00,sabato,1,2024,17,pomeriggio,inverno,True,Epifania del Signore
298,2023-05-07,20:30,Guardiani della Galassia vol.3,27.0,45.0,0.0,72.0,2023-05-07 20:30:00,domenica,5,2023,20,sera,primavera,True,Nessuna
283,2023-04-16,20:30,L'ultima notte d'amore,11.0,4.0,0.0,15.0,2023-04-16 20:30:00,domenica,4,2023,20,sera,primavera,True,Nessuna
201,2022-11-27,20:30,La Stranezza,27.0,17.0,0.0,44.0,2022-11-27 20:30:00,domenica,11,2022,20,sera,autunno,True,Nessuna


#### old weather script

In [4]:
# abilita progress_apply
tqdm.pandas(desc="Fetching weather")

def get_meteo(date, lat=46.0524, lon=11.45):
    url = (
        f"https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}&start_date={date}&end_date={date}"
        f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
        f"&timezone=Europe%2FRome"
    )
    resp = requests.get(url).json()
    if "daily" in resp:
        return {
            "temp_max": resp["daily"]["temperature_2m_max"][0],
            "temp_min": resp["daily"]["temperature_2m_min"][0],
            "precip_mm": resp["daily"]["precipitation_sum"][0]
        }
    else:
        return {"temp_max": None, "temp_min": None, "precip_mm": None}

# prepara le date
df["date_str"] = df["datetime"].dt.date.astype(str)

# usa progress_apply per la barra
meteo = df["date_str"].progress_apply(get_meteo)
meteo_df = pd.DataFrame(meteo.tolist())

# unisci
df = pd.concat([df, meteo_df], axis=1)


Fetching weather:  21%|██        | 125/604 [00:30<01:57,  4.09it/s]


ReadTimeout: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=None)

#### new one-block call 

In [25]:
# Prepara la colonna date_str
df["date_str"] = df["datetime"].dt.date.astype(str)
# Calcola start e end
start_date = df["date_str"].min()
end_date   = df["date_str"].max()

# Chiamata unica all’API per tutto l’intervallo
url = (
    "https://archive-api.open-meteo.com/v1/archive?"
    f"latitude=46.0524&longitude=11.45"
    f"&start_date={start_date}&end_date={end_date}"
    "&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
    "&timezone=Europe%2FRome"
)
resp = requests.get(url, timeout=30)
resp.raise_for_status()
data = resp.json()

# Costruisci il DataFrame meteo
meteo_df = pd.DataFrame({
    "date_str":        data["daily"]["time"],
    "temp_max":        data["daily"]["temperature_2m_max"],
    "temp_min":        data["daily"]["temperature_2m_min"],
    "precip_mm":       data["daily"]["precipitation_sum"]
})

# Unisci al DataFrame originale (left join per mantenere tutte le righe)
df = df.merge(meteo_df, on="date_str", how="left")

### TMDB call

In [26]:
TMDB_API_KEY = os.getenv("TMDB_API_KEY")
tqdm.pandas(desc="Fetching TMDb info")

def get_tmdb_info(title, year=None, api_key=TMDB_API_KEY):
    query = title.replace(" ", "+")
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={query}"

    try:
        res = requests.get(url)
        res.raise_for_status()
        results = res.json().get("results", [])
    except Exception as e:
        print(f"[✘] Errore nella richiesta per '{title}': {e}")
        return {
            "genres": None,
            "keywords": None,
            "cast": None,
            "director": None,
            "vote_average": None,
            "popularity": None
        }

    if not results:
        print(f"[✘] Titolo NON trovato: '{title}'")
        return {
            "genres": None,
            "keywords": None,
            "cast": None,
            "director": None,
            "vote_average": None,
            "popularity": None
        }

    # Se c'è un solo risultato, usalo direttamente
    if len(results) == 1:
        movie = results[0]
    else:
        # Più di un risultato → usa l'anno per scegliere il più vicino
        matches = [r for r in results if r.get("release_date", "").startswith(str(year))] if year else []
        movie = matches[0] if matches else results[0]

    movie_id = movie["id"]
    #print(f"[✓] Trovato: '{movie['title']}' → ID {movie_id} (anno: {movie.get('release_date')})")

    detail_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=credits,keywords"

    try:
        d = requests.get(detail_url).json()
    except Exception as e:
        print(f"[✘] Errore nella richiesta dettagliata per ID {movie_id}: {e}")
        return {
            "genres": None,
            "keywords": None,
            "cast": None,
            "director": None,
            "vote_average": None,
            "popularity": None
        }

    genres = [g["name"] for g in d.get("genres", [])]
    keywords = [k["name"] for k in d.get("keywords", {}).get("keywords", [])]
    cast = [c["name"] for c in d.get("credits", {}).get("cast", [])[:3]]
    director = next((c["name"] for c in d.get("credits", {}).get("crew", []) if c["job"] == "Director"), None)

    return {
        "genres": genres,
        "keywords": keywords,
        "cast": cast,
        "director": director,
        "vote_average": d.get("vote_average"),
        "popularity": d.get("popularity")
    }

tmdb_info = df.progress_apply(lambda row: get_tmdb_info(row["title"], row["anno"]), axis=1)
tmdb_df = pd.DataFrame(tmdb_info.tolist())
df = pd.concat([df.reset_index(drop=True), tmdb_df.reset_index(drop=True)], axis=1)

print(df)

# use the same input directory calculated at the beginning
output_path=os.path.join(os.path.dirname(file_path), "output", "cinema_data_updated.csv")

Fetching TMDb info:  33%|███▎      | 202/603 [00:47<01:23,  4.78it/s]

[✘] Titolo NON trovato: 'Tra i Due Mondi '


Fetching TMDb info:  70%|██████▉   | 422/603 [01:34<00:34,  5.28it/s]

[✘] Titolo NON trovato: 'Rimane il vento - Benvenuti in galera'


Fetching TMDb info:  70%|███████   | 423/603 [01:34<00:32,  5.59it/s]

...

[✘] Titolo NON trovato: 'Biancaneve v.o.'


Fetching TMDb info:  99%|█████████▉| 596/603 [02:12<00:01,  4.54it/s]

[✘] Titolo NON trovato: 'ANORA V.O.'


Fetching TMDb info: 100%|██████████| 603/603 [02:14<00:00,  4.48it/s]

[✘] Titolo NON trovato: 'QUEER V.O.'
          date   time               title  full_price  reduced  free  total  \
0   2019-09-14  20:30         Il Re Leone        83.0    101.0   0.0  184.0   
1   2019-09-15  17:00         Il Re Leone        53.0     67.0   1.0  121.0   
2   2019-09-15  20:30         Il Re Leone        35.0     22.0   0.0   57.0   
3   2019-09-22  17:00         Il Re Leone       114.0    127.0  13.0  254.0   
4   2019-09-28  20:30     IT - Capitolo 2        75.0    123.0   0.0  198.0   
..         ...    ...                 ...         ...      ...   ...    ...   
598 2025-05-02  20:30        THUNDERBOLTS         9.0     18.0   1.0   28.0   
599 2025-05-07  20:30               QUEER        18.0      9.0   0.0   27.0   
600 2025-05-10  20:30  THE LEGEND OF OCHI         9.0      7.0   0.0   16.0   
601 2025-05-11  17:00  THE LEGEND OF OCHI         3.0      4.0   1.0    8.0   
602 2025-05-11  20:30          QUEER V.O.        12.0     19.0   0.0   31.0   

              

In [31]:
# Ensure the output directory exists
output_dir = os.path.dirname(output_path)
os.makedirs(output_dir, exist_ok=True)

df.to_csv(os.path.join(output_dir, "dati_cinema_updated.csv"), index=False)